In [5]:
import pandas as pd
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV, GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from enum import Enum

from sklearn.linear_model import LogisticRegression

# Define an Enum class
class ML_TASK(Enum):
    CLASSIFICATION = 1
    REGRESSION = 2
    CLUSTERING = 3

class HYPERPARAM_SEARCH(Enum):
    RANDOM = 1
    GRID = 2
    HALVING_RANDOM = 3
    HALVING_GRID = 4

class DIM_REDUCTION_METHOD(Enum):
    PCA = 1
    KERNEL_PCA = 2
    ICA = 3

# Params
algo_name = 'LogisticRegression'
ml_task = ML_TASK.CLASSIFICATION # True if you want to map fractions to integers for classification
hyperparam_search_method = HYPERPARAM_SEARCH.GRID # True if you want to use Random Grid Search instead of brute force
dim_reduction_method = DIM_REDUCTION_METHOD.ICA # False if you don't want to use PCA
kfold_num = 10 # Train-Test-Split = 1, LOOCV = n

# Read
X_train = pd.read_csv('../pc_X_train.csv')
y_train = pd.read_csv('../pc_y_train.csv')
y_train = y_train.iloc[:, -1] # With iloc we extract the labels

X_test = pd.read_csv('../pc_X_test.csv')
ids = X_test.iloc[:, 0]

# If classification -> Do mapping
if ml_task == ML_TASK.CLASSIFICATION:
    possible_numbers = list(set(y_train))
    mapping = {val: idx for idx, val in enumerate(possible_numbers)}
    y_train = [mapping[val] for val in y_train]


In [6]:
# Normalize Trainset
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_train)

# PCA
dim_reduction = None
match dim_reduction_method:
    case DIM_REDUCTION_METHOD.PCA:
        dim_reduction = PCA()
    case DIM_REDUCTION_METHOD.KERNEL_PCA:
        dim_reduction = KernelPCA()
    case DIM_REDUCTION_METHOD.ICA:
        dim_reduction = FastICA()

# If classification
model = None
scoring_method = None
if ml_task == ML_TASK.CLASSIFICATION:
    model = LogisticRegression()
    scoring_method = 'f1_weighted'
    # raise ValueError("Logistic Regression is a regression algorithm. Please use ML_TASK.REGRESSION")
elif ml_task == ML_TASK.REGRESSION:
    scoring_method = 'neg_root_mean_squared_error'
    raise ValueError("Logistic Regression is a classification algorithm. Please use ML_TASK.CLASSIFICATION")

# Pipeline
steps = []
steps.append(('dim_reduction', dim_reduction))
steps.append(('model', model))

pipe = Pipeline(steps=steps)

# parameters = {'dim_reduction__
# n_components': [1,2,4,8,16], 
#               'model__criterion': ['gini', 'entropy', 'log_loss'], 
#               'model__splitter': ['best', 'random'],
#               'model__min_samples_split': [1,2,4,8,16,32],
#               'model__max_depth': [1,2,3,4,5,6,7,8,16,32,64]}

parameters = {'dim_reduction__n_components': [1,2,4,8,16,32],
              'model__penalty': ['l1', 'l2', 'elasticnet', None],
              'model__C': [0.5, 0.7, 1.0, 1.2, 1.4, 1.6, 2.0, 3.0],
              'model__random_state': [None],
              'model__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag','saga'],
              'model__max_iter': [1000]}

# Grid Search
grid_params_list = [pipe, parameters] 
grid_params_dict = {'cv': kfold_num, 'scoring': scoring_method, 'verbose':2, 'n_jobs':6}

grid = None
match hyperparam_search_method:
    case HYPERPARAM_SEARCH.GRID:
        grid = GridSearchCV(*grid_params_list, **grid_params_dict)
    case HYPERPARAM_SEARCH.RANDOM:
        grid = RandomizedSearchCV(*grid_params_list, **grid_params_dict)
    case HYPERPARAM_SEARCH.HALVING_GRID:
        grid = HalvingGridSearchCV(*grid_params_list, **grid_params_dict)
    case HYPERPARAM_SEARCH.HALVING_RANDOM:
        grid = HalvingRandomSearchCV(*grid_params_list, **grid_params_dict)

grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)

model = grid.best_estimator_

# Normalize Testset
X_normalized_test = scaler.fit_transform(X_test)
X_reduced_test = dim_reduction.fit_transform(X_normalized_test)

# Predict
predictions = model.predict(X_reduced_test)

# If classification -> Transform back to Regression Task
if ml_task == ML_TASK.CLASSIFICATION:
    predictions = [possible_numbers[val] for val in predictions]

# Save 
output_filename = f'bork_GYCAOB_{algo_name}_C.csv' if ml_task == ML_TASK.CLASSIFICATION else f'bork_GYCAOB_{algo_name}_R.csv'
results_df = pd.DataFrame({'id': ids,'score': predictions})
results_df.to_csv(output_filename, index=False)

Fitting 10 folds for each of 1152 candidates, totalling 11520 fits


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=l1, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=l1, model__random_state=None, model__solver=lbfgs; total time=   0.1s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=l1, model__random_state=None, model__solver=lbfgs; total time=   0.1s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=l1, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=l1, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=l1, model__random_state=None, model__solver=lbfgs; total time=   0.1s
[CV] END dim_reduction__n_componen

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None wil

[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction__n_components=1, model__C=0.5, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=lbfgs; total time=   0.2s
[CV] END dim_reduction

...

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
5280 fits failed out of a total of 11520.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
480 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *arg

[CV] END dim_reduction__n_components=32, model__C=3.0, model__max_iter=1000, model__penalty=None, model__random_state=None, model__solver=newton-cg; total time=  41.1s


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Parameters: {'dim_reduction__n_components': 32, 'model__C': 1.0, 'model__max_iter': 1000, 'model__penalty': 'l1', 'model__random_state': None, 'model__solver': 'saga'}
Best Score: 0.2918861917955291


/opt/homebrew/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but FastICA was fitted with feature names
  warnings.warn(
